Gaia Molinaro, April 2022, Berkeley (CA)

Going through the demo shown in https://payampiray.github.io/cbm by Payam Piray, but using Python with the MATLAB engine

The original repository is [here](https://github.com/payampiray/cbm). For the package to work with Python, a couple of changes are necessary (mainly for function calling). The adapted codes are in [this repository](https://github.com/gaiamolinaro/cbm).

In [13]:
import matlab.engine
import numpy as np
import matplotlib.pyplot as plt
from cbm_helpers import loadmat

In [15]:
m = matlab.engine.start_matlab()

# Initialize

In [ ]:
cbm_path = "C:/Program Files/MATLAB/R2022a/cbm-master"
m.addpath(cbm_path, nargout=0)

# Load the data

In [ ]:
data = m.load("example_RL_task/all_data.mat")["data"]
subj1 = data[0]

# Check model compatibility
Check that the models are compatible with cbm by generating a vector of random parameters and calling the llh funs
Note that in theory, RL models require constrained parameters (e.g. alpha between 0 and 1) while randn draws from
a random distribution. In model_RL and model_dualRL, some transformations applied to the normally-distributed
parameters to meet their theoretical bounds. If your models don't do so, you could try choosing parameters within
your range. F1 and F2 should be real negative scalars.
All models should have the structure: llh = model(parameters, data)

In [ ]:
parameters = m.randn(1, 2)
F1 = m.model_RL(parameters, subj1)
parameters = m.randn(1, 3)
F2 = m.model_dualRL(parameters, subj1)

# Fit individuals
First, we should run cbm_lap, which fits every model to each subject data separately (i.e. in a non-hierarchical
fashion). cbm_lap employs Laplace approximation, which needs a normal prior for every parameter.
We set zero as the prior mean. We also assume that the prior variance for all parameters is 6.25.
This variance is large enough to cover a wide range of parameters with no excessive penalty (see supplementary
materials of the reference article for more details on how this variance is calculated).

In [ ]:
v = 6.25  # same for all parameters
prior_RL = m.struct("mean", m.zeros(2, 1), "variance", v)  # note dimension of "mean" - corresponds to len(parameters)
prior_dualRL = m.struct("mean", m.zeros(3, 1), "variance", v)  # note dimension of "mean"

We also need to specify a file-address for saving the output of each model:

In [ ]:
fname_RL = "lap_RL.mat"
fname_dualRL = "lap_dualRL.mat"

Now we run cbm_lap for each model. Note that model_RL and model_dualRL are both in the current directory.

In [ ]:
m.cbm_lap(data, "model_RL", prior_RL, fname_RL, nargout=0)
m.cbm_lap(data, "model_dualRL", prior_dualRL, fname_dualRL, nargout=0)
# Running this command prints a report on your MATLAB output

Check the output file

In [ ]:
fname = m.load("lap_RL.mat")
cbm = fname["cbm"]

In [ ]:
# look at fitted parameters
np.array(cbm["output"]["parameters"])

# Hierarchical Bayesian Inference (HBI)

In [ ]:
# 1st input: data for all subjects (already defined above)
# data = m.load("example_RL_task/all_data.mat")["data"]

# 2nd input: a cell containing the models (function handles in the original)
models = m.cell(["model_RL", "model_dualRL"])

# 3rd input: another cell input containing file-address to files saved by cbm_lap
# note that they correspond to models (so pay attention to the order)
fcbm_maps = m.cell(["lap_RL.mat", "lap_dualRL.mat"])

# 4th input: a file address for saving the output
fname_hbi = "hbi_RL_dualRL.mat"

# This will throw an error saying ValueError: only a scalar struct can be returned from MATLAB
# but will still save the file
# Workaround: use an exception handling statement
try:
    m.cbm_hbi(data, models, fcbm_maps, fname_hbi, nargout=0)
    # Running this command prints a report on your MATLAB output
    fname_hbi = m.load("hbi_RL_dualRL.mat")
except ValueError:
    print("ValueError caught")
    fname_hbi = loadmat("hbi_RL_dualRL.mat")

cbm = fname_hbi["cbm"]

Check model frequency

In [ ]:
cbm["output"]["model_frequency"]

Check group means

In [ ]:
# group mean for parameters of model_RL
cbm["output"]["group_mean"][0]

In [ ]:
# group mean for parameters of model_dualRL
cbm["output"]["group_mean"][1]

Check parameters

In [ ]:
# RL
cbm["output"]["group_hierarchical_errorbar"][0]

In [ ]:
# dualRL
cbm["output"]["group_hierarchical_errorbar"][1]

# Plot
You can use the group_mean and group_hierarchical_errorbar values to plot group parameters,
or use cbm_hbi_plot to plot the main outputs of the HBI.

In [ ]:
# 1st input: the file-address of the file saved by cbm_hbi
fname_hbi = "hbi_RL_dualRL.mat"

# 2nd input: a cell input containing model names
model_names = m.cell(["RL", "Dual RL"])
# note that they correspond to models (so pay attention to the order)

# 3rd input: another cell input containing parameter names of the winning model
param_names = m.cell([r"\alpha^+", r"\alpha^-", r"\beta"])

# 4th input: another cell input containing transformation function associated with each parameter of the winning model
transform = m.cell(["sigmoid", "sigmoid", "exp"])
# note that if you use a less usual transformation function, you should pass the handle here (instead of a string)

# this function creates a model comparison plot (exceedanace probability and model frequency) as well as
# a plot of transformed parameters of the most frequent model.
k = int(np.argmax(cbm["output"]["model_frequency"])+1)   # model of interest (winning model) + 1 to account for 0-indexing in Python
save_figs = m.logical(0)  # set to 1 to save figures as png files
# important to give all arguments if using save_figs
m.cbm_hbi_plot(fname_hbi, model_names, param_names, transform, k, save_figs, nargout=0)

# Responsibility
We can look at the estimated responsibility that each model generated each individual dataset.
Across models, responsibilities sum to 1 for each subject.

In [ ]:
cbm["output"]["parameters"][0]

In [ ]:
cbm["output"]["parameters"][1]

In [ ]:
cbm["output"]["responsibility"]
# The first and second columns indicate the responsibility of model_RL and model_dualRL in generating the corresponding
# subject data, respectively.

Look at the estimated responsibility of model_dualRL:

In [ ]:
fig, ax = plt.subplots()
ax.plot(cbm["output"]["responsibility"][:, 1])
ax.set_ylim(-0.1, 1.1)
ax.set_title("model_dualRL Responsibility")

# Exceedance probability
The exceedance probability indicates the probability that each model is the most likely model across the group.

In [ ]:
cbm["output"]["exceedance_prob"]

# Protected exceedance probability
A more useful metric is called protected exceedance probability, which also takes into account the null hypothesis
that no model in the model space is most likely across the population (i.e. any difference between model frequencies
is due to chance).
This is currently only NaN values:

In [ ]:
cbm["output"]["protected_exceedance_prob"]

This is because for computing protected exceedance probabilities, the HBI should be re-run under the (prior) null
hypothesis.
This is how you can do it:

In [ ]:
# 1st input is data,
# 2nd input is the file-address of the file saved by cbm_hbi
try:
    m.cbm_hbi_null(data, fname_hbi)
except ValueError:
    print("ValueError caught")
cbm["output"]["protected_exceedance_prob"]

# Close the MATLAB engine

In [ ]:
m.exit()